In [1]:
import os
from covidanalytics import etl as ce
from covidanalytics import analytics as ca
from covidanalytics import plots as cp

In [2]:
reports_path=os.path.join(os.environ['HOME'],'Documents/code/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/')
html_path=os.path.join(os.environ['HOME'],'Documents/code/covid-static/pages/')

In [3]:
param_dict = {'reports_path': reports_path,
          'html': {'dir': html_path + 'bay_area.html',
                   'img': 'bay_area.png',
                   'img_shape': 'square'
                  },
          'query': 'Province_State=="California" & Admin2 in ["Alameda","Santa Clara","Santa Cruz","San Francisco","San Mateo","Marin","Contra Costa"]',
          'start_date': '03/22/2020',
          'end_date': '03/30/2020',
          'title': 'Bay Area - COVID-19'
         }

In [5]:
filenames = ce.dates_to_filenames(param_dict['start_date'], param_dict['end_date'])

In [16]:
recs = []
for f in filenames:
    try:
        f_path = param_dict['reports_path'] + f
        covid = ce.covid_csv_to_df(f_path)
        if 'state' in param_dict.keys():
            daily = ce.covid_province_state(covid, param_dict['state'])
        if 'country' in param_dict.keys():
            daily = ce.covid_country_region(covid, param_dict['country'])
        if 'query' in param_dict.keys():
            daily = ce.covid_query(covid, param_dict['query'])
        recs.append(daily)
    except:
        print(f)


In [7]:
f_path

'/Users/michaeln/Documents/code/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-30-2020.csv'

In [8]:
covid

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001,Abbeville,South Carolina,US,2020-03-30 22:52:45,34.223334,-82.461707,3,0,0,0,"Abbeville, South Carolina, US"
1,22001,Acadia,Louisiana,US,2020-03-30 22:52:45,30.295065,-92.414197,11,1,0,0,"Acadia, Louisiana, US"
2,51001,Accomack,Virginia,US,2020-03-30 22:52:45,37.767072,-75.632346,6,0,0,0,"Accomack, Virginia, US"
3,16001,Ada,Idaho,US,2020-03-30 22:52:45,43.452658,-116.241552,113,2,0,0,"Ada, Idaho, US"
4,19001,Adair,Iowa,US,2020-03-30 22:52:45,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"
...,...,...,...,...,...,...,...,...,...,...,...,...
3434,None,None,None,Venezuela,2020-03-30 22:52:33,6.423800,-66.589700,135,3,39,93,Venezuela
3435,None,None,None,Vietnam,2020-03-30 22:52:33,14.058324,108.277199,203,0,55,148,Vietnam
3436,None,None,None,West Bank and Gaza,2020-03-30 22:52:33,31.952200,35.233200,116,1,18,97,West Bank and Gaza
3437,None,None,None,Zambia,2020-03-30 22:52:33,-13.133897,27.849332,35,0,0,35,Zambia


In [9]:
'query' in param_dict.keys()

True

In [18]:
daily

{'query': 'Province_State=="California" & Admin2 in ["Alameda","Santa Clara","Santa Cruz","San Francisco","San Mateo","Marin","Contra Costa"]',
 'confirmed': 2138,
 'deaths': 52,
 'active': 2086,
 'date': '2020-03-30'}

In [19]:
recs

[{'query': 'Province_State=="California" & Admin2 in ["Alameda","Santa Clara","Santa Cruz","San Francisco","San Mateo","Marin","Contra Costa"]',
  'confirmed': 673,
  'deaths': 10,
  'active': 663,
  'date': '3/22/20'},
 {'query': 'Province_State=="California" & Admin2 in ["Alameda","Santa Clara","Santa Cruz","San Francisco","San Mateo","Marin","Contra Costa"]',
  'confirmed': 824,
  'deaths': 13,
  'active': 811,
  'date': '2020-03-23'},
 {'query': 'Province_State=="California" & Admin2 in ["Alameda","Santa Clara","Santa Cruz","San Francisco","San Mateo","Marin","Contra Costa"]',
  'confirmed': 978,
  'deaths': 20,
  'active': 958,
  'date': '2020-03-24'},
 {'query': 'Province_State=="California" & Admin2 in ["Alameda","Santa Clara","Santa Cruz","San Francisco","San Mateo","Marin","Contra Costa"]',
  'confirmed': 1122,
  'deaths': 26,
  'active': 1096,
  'date': '2020-03-25'},
 {'query': 'Province_State=="California" & Admin2 in ["Alameda","Santa Clara","Santa Cruz","San Francisco","S

In [20]:
covid_ts = ca.covid_dict_to_timeseries_df(recs, param_dict['start_date'], param_dict['end_date'])


ValueError: cannot convert float NaN to integer

In [26]:
import pandas as pd
import numpy as np

In [23]:
df=pd.DataFrame(recs,index=pd.date_range('03/22/2020','03/30/2020'))

In [24]:
df

,query,confirmed,deaths,active,date
2020-03-22,"Province_State==""California"" & Admin2 in [""Ala...",673,10,663,3/22/20
2020-03-23,"Province_State==""California"" & Admin2 in [""Ala...",824,13,811,2020-03-23
2020-03-24,"Province_State==""California"" & Admin2 in [""Ala...",978,20,958,2020-03-24
2020-03-25,"Province_State==""California"" & Admin2 in [""Ala...",1122,26,1096,2020-03-25
2020-03-26,"Province_State==""California"" & Admin2 in [""Ala...",1354,31,1323,2020-03-26
2020-03-27,"Province_State==""California"" & Admin2 in [""Ala...",1556,34,1522,2020-03-27
2020-03-28,"Province_State==""California"" & Admin2 in [""Ala...",1663,42,1621,2020-03-28
2020-03-29,"Province_State==""California"" & Admin2 in [""Ala...",1816,46,1770,2020-03-29
2020-03-30,"Province_State==""California"" & Admin2 in [""Ala...",2138,52,2086,2020-03-30


In [28]:
df['total_change'] = np.zeros((len(df.index),1))
df

,query,confirmed,deaths,active,date,total_change
2020-03-22,"Province_State==""California"" & Admin2 in [""Ala...",673,10,663,3/22/20,0.0
2020-03-23,"Province_State==""California"" & Admin2 in [""Ala...",824,13,811,2020-03-23,0.0
2020-03-24,"Province_State==""California"" & Admin2 in [""Ala...",978,20,958,2020-03-24,0.0
2020-03-25,"Province_State==""California"" & Admin2 in [""Ala...",1122,26,1096,2020-03-25,0.0
2020-03-26,"Province_State==""California"" & Admin2 in [""Ala...",1354,31,1323,2020-03-26,0.0
2020-03-27,"Province_State==""California"" & Admin2 in [""Ala...",1556,34,1522,2020-03-27,0.0
2020-03-28,"Province_State==""California"" & Admin2 in [""Ala...",1663,42,1621,2020-03-28,0.0
2020-03-29,"Province_State==""California"" & Admin2 in [""Ala...",1816,46,1770,2020-03-29,0.0
2020-03-30,"Province_State==""California"" & Admin2 in [""Ala...",2138,52,2086,2020-03-30,0.0


In [29]:
df['percent_change'] = np.zeros((len(df.index),1))
df

,query,confirmed,deaths,active,date,total_change,percent_change
2020-03-22,"Province_State==""California"" & Admin2 in [""Ala...",673,10,663,3/22/20,0.0,0.0
2020-03-23,"Province_State==""California"" & Admin2 in [""Ala...",824,13,811,2020-03-23,0.0,0.0
2020-03-24,"Province_State==""California"" & Admin2 in [""Ala...",978,20,958,2020-03-24,0.0,0.0
2020-03-25,"Province_State==""California"" & Admin2 in [""Ala...",1122,26,1096,2020-03-25,0.0,0.0
2020-03-26,"Province_State==""California"" & Admin2 in [""Ala...",1354,31,1323,2020-03-26,0.0,0.0
2020-03-27,"Province_State==""California"" & Admin2 in [""Ala...",1556,34,1522,2020-03-27,0.0,0.0
2020-03-28,"Province_State==""California"" & Admin2 in [""Ala...",1663,42,1621,2020-03-28,0.0,0.0
2020-03-29,"Province_State==""California"" & Admin2 in [""Ala...",1816,46,1770,2020-03-29,0.0,0.0
2020-03-30,"Province_State==""California"" & Admin2 in [""Ala...",2138,52,2086,2020-03-30,0.0,0.0


In [30]:
final = df['confirmed'].loc[df.index[1]:df.index[-1]].values
final

array([ 824,  978, 1122, 1354, 1556, 1663, 1816, 2138])

In [31]:
initial = df['confirmed'].loc[df.index[0]:df.index[-2]].values
initial

array([ 673,  824,  978, 1122, 1354, 1556, 1663, 1816])

In [35]:
tc = final - initial
tc

array([151, 154, 144, 232, 202, 107, 153, 322])

In [36]:
tc = np.insert(tc,0,0)
tc

array([  0, 151, 154, 144, 232, 202, 107, 153, 322])

In [37]:
df['total_change'] = tc
df

,query,confirmed,deaths,active,date,total_change,percent_change
2020-03-22,"Province_State==""California"" & Admin2 in [""Ala...",673,10,663,3/22/20,0,0.0
2020-03-23,"Province_State==""California"" & Admin2 in [""Ala...",824,13,811,2020-03-23,151,0.0
2020-03-24,"Province_State==""California"" & Admin2 in [""Ala...",978,20,958,2020-03-24,154,0.0
2020-03-25,"Province_State==""California"" & Admin2 in [""Ala...",1122,26,1096,2020-03-25,144,0.0
2020-03-26,"Province_State==""California"" & Admin2 in [""Ala...",1354,31,1323,2020-03-26,232,0.0
2020-03-27,"Province_State==""California"" & Admin2 in [""Ala...",1556,34,1522,2020-03-27,202,0.0
2020-03-28,"Province_State==""California"" & Admin2 in [""Ala...",1663,42,1621,2020-03-28,107,0.0
2020-03-29,"Province_State==""California"" & Admin2 in [""Ala...",1816,46,1770,2020-03-29,153,0.0
2020-03-30,"Province_State==""California"" & Admin2 in [""Ala...",2138,52,2086,2020-03-30,322,0.0


In [39]:
denom = initial[np.where(initial==0)]=np.nan


ValueError: cannot convert float NaN to integer

In [38]:
np.where(initial==0)

(array([], dtype=int64),)

In [40]:
denom = initial[[]]=np.nan

ValueError: cannot convert float NaN to integer

In [41]:
idx=np.where(initial==0)

In [44]:
idx[0].size

0